# Turkish ULMFiT from scratch

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
# bs=128
bs=48
# torch.cuda.set_device(2)
torch.cuda.set_device(0)
data_path = Config.data_path()

lang = 'tr'
name = f'{lang}wiki'
path = data_path/name

path.mkdir(exist_ok=True, parents=True)

In [3]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

## Turkish wikipedia model

In [4]:
%%time
# excludes articles with fewer than 1800 words, since they tend to be "stubs" with mostly boilerplate.
from nlputils import split_wiki,get_wiki

Wall time: 999 µs


In [21]:
get_wiki(path,lang)

extracting...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\cross-entropy\\.fastai\\data\\trwiki\\text\\AA\\wiki_00'

#### Download failed but provided a helpful error message!
Download of https://dumps.wikimedia.org/trwiki/latest/trwiki-latest-pages-articles.xml.bz2 has failed after 5 retries

We'll install the files manually, following instructions provided in the error message (see below).


#### Windows 10 users, follow these steps if you have not already done so:

1) Follow the instructions here to install `Chocolatey`:
https://chocolatey.org/install

2) In an `administrator PowerShell`, install `wget` version 1.20 (or later?) via this command:

`choco install wget --version 1.20`

3) Download and install 7-zip for Windows 10: https://www.7-zip.org/download.html


#### Fix the download manually:
 
\$ mkdir -p C:\Users\cross-entropy\.fastai\data

\$ cd C:\Users\cross-entropy\.fastai\data

\$ wget https://dumps.wikimedia.org/trwiki/latest/trwiki-latest-pages-articles.xml.bz2

Use 7-zip to extract trwiki-latest-pages-articles.xml.bz2

#### Directly download trwiki-latest-pages-articles.xml.bz2 from https://archive.org/details/wikipediadumps

Then extract it with the following code, snipped from nlputils.py

In [6]:
xml_fn = f"{lang}wiki-latest-pages-articles.xml"
zip_fn = f"{xml_fn}.bz2"

if not (path/xml_fn).exists():
    print("downloading...")
    download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
    print("unzipping...")
    bunzip(path/zip_fn)
print(xml_fn)

trwiki-latest-pages-articles.xml


In [7]:
!head -n100 {path}/trwiki-latest-pages-articles.xml


<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.4/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.4/ http://www.mediawiki.org/xml/export-0.4.xsd" version="0.4" xml:lang="tr">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <base>http://tr.wikipedia.org/wiki/Ana_Sayfa</base>
    <generator>MediaWiki 1.16alpha-wmf</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2">Medya</namespace>
      <namespace key="-1">Ã–zel</namespace>
      <namespace key="0" />
      <namespace key="1">TartÄ±ÅŸma</namespace>
      <namespace key="2">KullanÄ±cÄ±</namespace>
      <namespace key="3">KullanÄ±cÄ± mesaj</namespace>
      <namespace key="4">Vikipedi</namespace>
      <namespace key="5">Vikipedi tartÄ±ÅŸma</namespace>
      <namespace key="6">Dosya</namespace>
      <namespace key="7">Dosya tartÄ±ÅŸma</namespace>
      <namespace key="8">MedyaViki</namespace>
      <namespace key="9">MedyaViki

In [8]:
(path/xml_fn).exists()

True

#### Use WikiExtractor to process the .xml file

In [7]:
xml_fn_full = path/xml_fn
print(xml_fn_full)
xml_fn_full.exists()

C:\Users\cross-entropy\.fastai\data\trwiki\trwiki-latest-pages-articles.xml


True

In [8]:
path

WindowsPath('C:/Users/cross-entropy/.fastai/data/trwiki')

In [9]:
path_nlp = Path(r'C:\Users\cross-entropy\fastai\course-nlp')
print(path_nlp)

C:\Users\cross-entropy\fastai\course-nlp


In [10]:
pp = path_nlp.joinpath(r'C:\Users\cross-entropy\fastai\course-nlp\text\AA\wiki_00')

In [11]:
pp

WindowsPath('C:/Users/cross-entropy/fastai/course-nlp/text/AA/wiki_00')

In [12]:
pp.exists()

True

In [13]:
path_nlp/'text/AA/wiki_00'

WindowsPath('C:/Users/cross-entropy/fastai/course-nlp/text/AA/wiki_00')

In [14]:
str(path_nlp/'text/AA/wiki_00')

'C:\\Users\\cross-entropy\\fastai\\course-nlp\\text\\AA\\wiki_00'

In [15]:
str(path/name)

'C:\\Users\\cross-entropy\\.fastai\\data\\trwiki\\trwiki'

In [16]:
name

'trwiki'

In [17]:
path/name

WindowsPath('C:/Users/cross-entropy/.fastai/data/trwiki/trwiki')

In [18]:
path/'text'

WindowsPath('C:/Users/cross-entropy/.fastai/data/trwiki/text')

In [23]:
import os
os.system("python C:/Users/cross-entropy/wikiextractor/WikiExtractor.py --processes 4 --no_templates " +
    f"--min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q {xml_fn_full}")


1

In [24]:
shutil.move(str(path_nlp/'text/AA/wiki_00'), str(path/name))
shutil.rmtree(path/'text')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\cross-entropy\\fastai\\course-nlp\\text\\AA\\wiki_00'

In [27]:
dest = split_wiki(path,lang)

C:\Users\cross-entropy\.fastai\data\trwiki\docs already exists; not splitting


Turkish is an [Agglutinative_language](https://en.wikipedia.org/wiki/Agglutinative_language) so it needs special care!

![Turkish morphemes example](images/turkish.jpg)

In [18]:
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor()]))

In [19]:
data

TextList (0 items)

Path: C:\Users\cross-entropy\.fastai\data\trwiki\docs

In [62]:
path2 = path/'trwiki-latest-pages-articles.xml.bz2'

In [63]:
path2

WindowsPath('C:/Users/cross-entropy/.fastai/data/trwiki/trwiki-latest-pages-articles.xml.bz2')

In [64]:
!dir {path2}

 Volume in drive C is OS
 Volume Serial Number is 5A82-462F

 Directory of C:\Users\cross-entropy\.fastai\data\trwiki

02/21/2020  12:31 PM        19,922,944 trwiki-latest-pages-articles.xml.bz2
               1 File(s)     19,922,944 bytes
               0 Dir(s)  172,161,990,656 bytes free


In [9]:
f'{lang}_databunch'

'tr_databunch'

#### Use `sentencepiece` to tokenize into sub-word units, with a vocab of 30,000 words

In [16]:
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor()])
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

RuntimeError: Internal: C:\projects\sentencepiece\src\trainer_interface.cc(49) [(trainer_spec.vocab_size()) > (0)] 

In [ ]:
data = load_data(dest, f'{lang}_databunch', bs=bs)

In [12]:
data.show_batch()

idx,text
0,"▁ele ▁geçirerek ▁politik ▁bir ▁özerklik ▁verdiğini ▁belirtmek le ▁birlikte ▁xxmaj ▁tibet ' in ▁yalnızca ▁1913 - 1950 ▁yılları ▁arasında ▁xxmaj ▁çin ' in ▁politik ▁nüfuz undan ▁çıktığını , ▁bölgenin ▁tarihi ▁olarak ▁xxmaj ▁çin ' e ▁ait ▁olduğunu ▁düşünmektedir . ▁xxmaj ▁tibet ' in ▁kendi ▁kültür ▁ve ▁zenginlik lerinin ▁"" kültürel ▁bir ▁soykırım "" a ▁tabi ▁tutulduğu ▁da ▁iddialar ▁arasındadır . ▁xxmaj ▁çin ▁hükümeti ▁ise ▁bu ▁"" kültürel ▁soykırım "" ▁iddia"
1,"▁dışı ▁kalan ▁olmasa ▁da , ▁o ▁yarışta ▁lastik ▁üreticisi ▁xxmaj ▁michel in ' le ▁yaşanan ▁problemler den ▁dolayı ▁güvenlik ▁gerekçesiyle ▁xxmaj ▁michel in ▁lastik leri ▁kullanan ▁tüm ▁takımlar ▁çekilmiş , ▁yarış a ▁sadece ▁6 ▁pilot ▁katılmıştır . ▁xxmaj ▁bunların ▁dışında ▁xxmaj ▁ ant ô ni o ▁xxmaj ▁pi zz onia ▁xxmaj ▁i ̇ talya ▁ve ▁sonrasında ▁sezonun ▁geri ▁kalanında ▁xxmaj ▁nick ▁xxmaj ▁he id feld ' in ▁yerine ▁yarışmıştır ."
2,"iye ▁oyunlarından ▁çok ▁farklı ▁bulduğunu ▁belirterek ▁taz iye yi ▁de ▁içinde ▁barındıran ▁benzersiz ▁bir ▁yapıt ▁olduğunu ▁yazdı . ▁< ▁/ ▁doc > ▁xxbos ▁xxmaj ▁anahtar sız ▁şifreleme ▁xxmaj ▁anahtar sız ▁şifreleme , ▁anahtar ▁kullanmaya n ▁kriptografik ▁algoritma lar , ▁veya ▁diğer ▁adlarıyla ▁xxmaj ▁veri ▁xxmaj ▁bütünlüğü ▁ve ▁xxmaj ▁özet ▁xxmaj ▁fonksiyonları , ▁veri ▁bütünlüğünü ▁garanti ▁etmek ▁için ▁kullanılan ▁xxup ▁md 5 , ▁xxup ▁sha -1 , ▁xxup ▁ rip em"
3,"rib ine ▁bağlı ▁olarak ▁akarsu ▁havza larında ▁artan ▁sediman ▁üretimi ▁f lü v yal ▁sistemlerde ▁etkili ▁olmuştur . ▁20 ▁yy ’ ın ▁ikinci ▁yarısından ▁itibaren ▁akarsular ▁üzerinde ▁çok ▁sayıda ▁baraj ▁yapılmıştır . ▁xxmaj ▁baraj ▁yapımı ▁nedeniyle , ▁akarsu nun ▁taşıdığı ▁sediman ▁miktarı ▁çevre nin ▁tahrip ▁edilmesi ▁nedeniyle ▁artmaktadır . ▁xxmaj ▁akarsu ▁yatakları ndan ▁kum ▁alınması , ▁akarsu nun ▁yatağı nın ▁yeniden ▁düzenlemesi ne ▁neden ▁olmaktadır . ▁xxmaj ▁teras lar ;"
4,"n ür lük "", ▁tek ▁başına ▁sensör ▁çözünürlüğü ne ▁bağlı ▁değildir . ▁"" mak sim um ▁sistem ▁çözünürlüğü "", ▁diğer ▁etkenler ▁( sen s ör ▁üzerinde ▁yer ▁alan ▁filtreler , ▁kullanılan ▁objektif in ▁optik ▁kalitesi ▁ve ▁diyafram ▁açıklığı ▁vb . ) ▁dışarıda ▁tutulur sa ▁aşağıdaki ▁formül ▁ile ▁hesaplanabilir ▁( birim ▁= ▁xxup ▁lp ▁/ ▁xxup ▁mm ▁xxmaj ▁i ̇ ng . : ▁ line ▁pa ir s ▁per ▁mili meter"


In [13]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [14]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [15]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.337346,4.472464,0.297502,10:04
1,4.145160,4.359449,0.304195,10:29
2,4.199796,4.386935,0.301542,10:29
3,4.132432,4.329662,0.305723,10:30
4,4.048324,4.241048,0.313223,10:32
5,3.943493,4.118834,0.323640,10:30
6,3.857112,3.969563,0.337475,10:30
7,3.695721,3.799551,0.354562,10:30
8,3.602813,3.652005,0.371855,10:31
9,3.519019,3.596706,0.379076,10:31


In [32]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [24]:
path_clas = path/'movies'
path_clas.ls()

[PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.pos'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_clas_databunch'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/models'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.neg'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_data_lm')]

In [25]:
# could not read files encoded in UTF-8; Jeremy found that `iso-8859-9` was often used for Turkish language encoding
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,gerçekten harika bir yapim birçok kez izledim ...,1
1,her izledigimde hayranlik duydugum gerçek klas...,1
2,gerçekten tarihi savas filmleri arasinda tarti...,1
3,aldigi ödülleri sonuna dek hak eden muhtesem b...,1
4,özgürlük denilince aklima gelen ilk film.bir b...,1


In [26]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,giseye oynayan bir film.mel gibson'in oyunculu...,0
1,bircok yonden sahip olduklari zayifliklari pop...,0
2,"1995 ten bu yana bu tür filmler artti , o zama...",0
3,mel gibson tam bir ingiliz düsmani her filmind...,0
4,milliyetçi bir film tavsiye etmiyorum.... \n,0


In [27]:
# combine train and test data since language model doesn't use the labels!
df = pd.concat([pos_df,neg_df], sort=False)

In [28]:
# use same `sentencepiece` processor and vocab
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

data_lm.save(f'{lang}_clas_databunch')

In [29]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [30]:
data_lm.show_batch()

idx,text
0,"▁bile ▁sinema ▁olgusu nun ▁en ▁üst ▁noktalar in dan . . ▁xxbos ▁gerçekten ▁tarihi ▁sava s ▁filmleri ▁ara si nda ▁tar tis ma siz ▁en ▁iyi si ▁ , ▁12 ▁ yi l ▁boyunca ▁ac aba ▁ikincisi ▁çek ir imi ▁diye ▁bekledi gi m ▁bir ▁film ▁ , bel ki ▁william ▁wallace ▁baba sinin ▁ölümünden ▁sonra ▁amca si ▁yani na ▁al m isti ▁onu ▁ ye tis tir m isti"
1,ka dra j lar i yla ▁konusu yla ▁is le ni siyle ▁insana ▁ iste ▁film ▁böyle ▁çekilir ▁de dir ten ▁kusur suz ▁bir ▁film . ▁xxbos ▁böyle ▁güzel ▁bir ▁yap it ▁olamaz ▁filmde ▁her ▁sey ▁var ▁insani ▁dünya dan ▁ali p ▁gö tür üyor ▁bask a ▁diyar lara ▁film ▁bitti kten ▁sonra ▁epey ▁süre ▁geçmesi ▁gerekiyor ▁tekrar ▁dünya ▁ya ▁dönmek ▁için ▁dikkat ! . ▁xxbos ▁inan ir mi siniz
2,▁o ▁kadar ▁etki len mis tim ▁ki . . . özellikle ▁özgürlük ▁ ug ru na ▁sava san ▁william ▁wallace  in ▁is ken ce ▁edilerek ▁idam ▁edilmesi . . . ve ▁sonunda ▁özgürlük ▁diye ▁hay kir isi . . . ha lan ▁unut ami yorum ▁xxrep ▁4 ▁ . ▁xxbos ▁ilk ▁bu ▁filmi ▁sinema da ▁izledi m ▁ve ▁insan in ▁inan di ktan ▁sonra ▁ ne leri ▁yap a
3,▁için ▁ister ▁istemez ▁oraya ▁gö tür üyor ▁filmin ▁uzun lu gun a ▁al dan ip ta ▁filmi ▁izlemek ten ▁vazgeçme yin ▁xxrep ▁4 ▁ . ▁xxbos ▁harika ▁bir ▁film di ▁xxrep ▁5 ▁ . ▁xxbos ▁mükemmel ▁ötesi . . ▁ . ▁xxbos ▁hiç ▁ a bart mi yorum ▁hayat im da ▁izledi gi m ▁en ▁iyi ▁film lerden ▁biri ▁diye bilir im . tam ▁bir ▁bas yap it ▁nite ligi
4,les iyor . ▁herkes ▁izleme li . . . ▁xxbos ▁tarantino nun ▁en ▁iyi ▁filmlerinden ▁biri . ▁diyalog lar ▁çok ▁iyi . ▁kesinlikle ▁izlenmesi ▁gereken ▁bir ▁film . . ▁xxbos ▁tarantino nun ▁bu ▁filmi ▁kendini ▁belli ▁etti r iyor . hat ta ▁ben ce ▁tarantino nun ▁en ▁iyi ▁filmidir . kendi ne ▁has ▁anlat imi ▁ile ▁bu ▁film ▁hak ka ten ▁sinema ▁sever lerin ▁izleme si ▁gereken ▁bir ▁film .


In [31]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [32]:
lr = 1e-3
lr *= bs/48

In [33]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.762205,4.132825,00:07


In [34]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.136730,3.976010,00:08
1,3.991336,3.882426,00:08
2,3.768218,3.812365,00:08
3,3.526307,3.795765,00:08
4,3.310204,3.815228,00:08


In [35]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [37]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [38]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [39]:
lr=2e-2
lr *= bs/48

In [40]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.456195,0.441739,0.817073,00:02
1,0.416383,0.359390,0.841463,00:02


In [41]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.413642,0.381773,0.828330,00:02
1,0.407912,0.358319,0.846154,00:02


In [42]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.358337,0.310041,0.869606,00:02
1,0.279921,0.276278,0.892120,00:02


In [43]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.258410,0.261940,0.899625,00:03
1,0.183993,0.267610,0.905253,00:02


In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.115543,0.290134,0.902439,00:03
1,0.107257,0.297127,0.911820,00:03


Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [158]:
learn_c.save(f'{lang}clas')

## fin